## Presentation Types with Hidden Types
## Batch Output to CSV in Sets

In [18]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [6]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece ID here
mei_file = 'CRIM_Model_0001.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Model_0001.mei
{'title': 'Veni speciosam', 'composer': 'Lupi, Johannes'}


## Run the Classifier Here

- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



### Run Classifier on Several Pieces at Once

Results are combined into a single dataframe

In [13]:


git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')
piece_list =  ['CRIM_Mass_0005_3.mei',
             'CRIM_Mass_0005_4.mei',
             'CRIM_Mass_0005_5.mei',
             'CRIM_Model_0001.mei',
             'CRIM_Mass_0002_1.mei',
             'CRIM_Mass_0002_2.mei',
             'CRIM_Mass_0002_3.mei',
             'CRIM_Mass_0002_4.mei',
             'CRIM_Mass_0002_5.mei',
             'CRIM_Model_0015.mei',
             'CRIM_Mass_0013_1.mei',
             'CRIM_Mass_0013_2.mei',
             'CRIM_Mass_0013_3.mei',
             'CRIM_Mass_0013_4.mei',
             'CRIM_Mass_0013_5.mei',
             'CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']


In [2]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

piece_list = ['CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']

In [14]:
include_hidden_types = False
melodic_ngram_length = 4
edit_distance_threshold = 0
final = pd.DataFrame()
for work in piece_list:
    url = git_prefix + work
    piece = importScore(url)   
    nr = piece.getNoteRest(combineUnisons=True)
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
    mel = piece.getMelodic(df=nr)
    mel_ng = piece.getMelodicEntries(interval_settings=('d', True, True), n=melodic_ngram_length)
    output = classify_entries_as_presentation_types(piece, nr, dur_ng, mel_ng, edit_distance_threshold, include_hidden_types)
    final = final.append(output, ignore_index=True)
final

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[-3, 3, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,7,False
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[-3, 2, 2, -2]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8,False
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 40/2.0, 42/2.0, 44/1.0, 46/2....","[P-5, P8, P-12, P8, P8, P-11]","[298.0, 302.0, 314.0, 330.0, 344.0, 362.0, 382.0]","[1, 1, -3, 3]","[4.0, 12.0, 16.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Cantus, Bassus, Altus, Cantus, ...",FUGA,7,False
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[-2, 2, 1, 3]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9,False
4,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...
56,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,280.0,"[35/1.0, 38/1.0]",[P-8],"[280.0, 304.0]","[-3, 3, 2, -2]",[24.0],"[Cantus, Quinta Pars]",FUGA,2,False
57,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,332.0,"[41/3.0, 43/2.0, 44/3.0, 46/2.0, 48/4.0]","[P-5, P-4, P4, P-8]","[332.0, 346.0, 356.0, 370.0, 390.0]","[1, -3, 1, 2]","[14.0, 10.0, 14.0, 20.0]","[Cantus, Altus, Quinta Pars, Tenor, Bassus]",FUGA,5,False
58,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,404.0,"[50/3.0, 53/3.0]",[P-8],"[404.0, 428.0]","[1, 1, -3, 3]",[24.0],"[Cantus, Quinta Pars]",FUGA,2,False
59,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/4.0, 59/4.0, 61/4.0, 64/4.0]","[P1, P-8, P11, P-8]","[446.0, 454.0, 478.0, 494.0, 518.0]","[-2, 2, 1, 3]","[8.0, 24.0, 16.0, 24.0]","[Altus, Cantus, Quinta Pars, Cantus, Quinta Pars]",FUGA,5,False


In [15]:
final.to_csv("saved_csv/Mass_0019_PTsimple.csv")